# HW2: Part 1 - Primal-dual Interior-point method

In [9]:
import numpy as np, pandas as pd, scipy.optimize as sopt, sympy as sp
import matplotlib.pyplot as plt
import time
import math
from math import gcd
from fractions import Fraction
from functools import reduce
from scipy.linalg import lu

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## 1. Method implementation

In [31]:
class Observer:
    """
    Log key statistics and save them to a file for analysis (not used for this assignment).
    str: name:
    str: savefile:
    str: observations:
    str: eval_fun:
    str: run: run id
    """
    def __init__(self, eval_fun, name):
        self.name = name
        self.savefile = name + ".tex"
        self.observations = pd.DataFrame(columns=["fun", "run", "time", "iter", "score"])
        self.eval_fun = eval_fun
        self.run = 0
        
    def start_observing(self):
        self.stime = time.time()
        self.niter = 0
        self.run += 1
        
    def observe(self, citer, x):
        self.observations = self.observations.append({"fun": self.name,
                                                      "run": self.run,
                                                      "time": time.time() - self.stime,
                                                      "iter": citer,
                                                      "x": x}, ignore_index = True)
    
    def evaluate(self):
        self.observations["score"] = self.observations["x"].apply(self.eval_fun)
    
    def save_observations(self):
        self.observations.to_latex(self.savefile)

class InteriorPoint:
    """
    Dual primal IP method utilizing big M method for initial solution. 
    """
    def __init__(self, obs = None, lower_bnd = None, upper_bnd = None, maxiter = 100000):
        """
        Observer:: obs: logger for our algorithm
        float:: lower_bnd: setable lower bound for big M method, precision of IP solution 
                (if None calculate using rules specified in report)
        float:: upper_bnd: setable upper bound for big M method, precision of IP solution 
                (if None calculate using rules specified in report)
        int:: maxiter: maximum number of iterates our algorithm can use during its run
        boolean:: stop_flag: indicates ocurrence of error
        str:: error_message: empty if no error otherwise specifies encountered error
        """
        self.lower_bnd = lower_bnd
        self.upper_bnd = upper_bnd
        self.observer = obs
        self.maxiter = maxiter
        
        self.stop_flag = False
        self.error_mesage = None
        
    class Result:
        """
        Wrapper class used to return all relevant information to the user about the method run.
        """
        def __init__(self, x_opt, cost, is_feasible, is_bounded, y_opt=None, s_opt=None):
            """
            np.array[float]:: x_opt: found optimal solution 
                                    (trimmed doesn't include slack or big M additional elements)
            float:: cost: calculated cost for x
            bolean:: is_feasible: flag indicating wether our problem is feasible
            boolean:: is_bounded: flag indicating wether our problem is bounded
            y_opt, s_opt:: np.array[float]: used exclusively for finding optimum in underdetermined problem
            """
            self.x = x_opt
            self.cost = cost
            self.feasible = is_feasible
            self.bounded = is_bounded
            
            self.y = y_opt
            self.s = s_opt
    
    def make_full_rank(self, A, b):
        """
        Make rows of matrix [A | b] independent (using numpy.linalg.lu).
        """
        # make sure b column vector
        b = b if b.shape[1] == 1 else b.T
        
        Ab = np.c_[A, b]
        U = lu(Ab.T)[2].T
        lin_indep_rows = [i for i in range(U.shape[0]) if any(U[i, :] != 0)]
        return A[lin_indep_rows, :], b[lin_indep_rows, :]
    
    def get_max(self, A, b):
        """
        Get maximum x_i possible for Ax = b.
        Rules for determining x_i:
            - all a_i in row positive, then x_i <= b / a_i
            - a_i = 0,                 then a_i = inf
            - any a_i in row negative (limit) check limits from other rows if all x_i where a_i limited
                    then calculate b_prime = b - max_i (a_i * x_i forall i where a_i < 0) then calculate remaining x_i bound
            - any a_i in row negative check limits from other rows if not all x_i where a_i limited then all x_i can be inf in row
        (This heuristic gives much better upper bounds for diet problem upper bound given by article > 1e40, by heuristic = 2400, furthermore
            this heuristic works in general case when coefficients not integral additionally if max x_i = inf problem unbounded.)
        """
        
        B = np.asmatrix(b) * np.asmatrix(np.ones(A.shape[1])) # B = [b b b ... b]
        max_coeff = np.asmatrix(np.zeros(A.shape)) # initialize X coeff matrix
        
        # case where b_i and a_i positive (x_i = b_i / a_i)
        all_pos = (B > 0) & (A > 0) 
        max_coeff[np.where(all_pos)] = np.divide(B[all_pos], A[all_pos])
        
        # case where a_i = 0 , x_i = inf
        max_coeff[(B == 0) & (A == 0)] = float("inf")
        max_coeff[(B > 0) & (A == 0)] = float("inf")
        
        # case where any a_i < 0, bounding from other rows neccesary
        A_neg_any = np.asmatrix(np.any(A < 0, axis=1)) * np.asmatrix(np.ones(A.shape[1])).astype(bool)
        max_coeff[A_neg_any] = float("inf") # initialize row to inf
        ub = np.min(max_coeff, axis=0) # get current x_i upper bounds
        with np.printoptions(precision=1, linewidth=150, suppress=True):
            nlb = np.nanmin(np.multiply(A, (np.asmatrix(np.ones(A.shape[0])).T * ub)), axis=1) # min_i a_i * x_i for a_i < 0 
            NB = B.copy() # copy neccesary to not rewrite original B matrix
            NB = (B - (nlb * np.asmatrix(np.ones(A.shape[1]))))  # NB new b with nb = b - min a_i * x for a_i < 0
            
            # repeat same process for new b
            some_pos = (NB > 0) & (A > 0) & (A_neg_any) 
            max_coeff[np.where(some_pos)] = np.divide(NB[some_pos], A[some_pos])
            max_coeff[(NB <= 0) & (A > 0) & (A_neg_any)] = 0 # a_i > 0 and b < 0 change inf -> 0 not neccesary above because everything initialized to zero

        W = np.max(np.min(max_coeff, axis=0))
        return W
    
    def get_min(self, A, b):
        """
        Get minimum x_i possible for Ax = b.
        Rules for determining x_i:
            - all a_i in row negative, then x_i >= b / a_i
            - a_i = 0,                 then a_i -> 0
            - any a_i in row positive (limit) check limits from other rows if all x_i where a_i limited
                    then calculate b_prime = b - min_i (a_i * x_i forall i where a_i > 0) then calculate remaining x_i bound
            - any a_i in row positive check limits from other rows if not all x_i where a_i limited then all x_i can be infinitesimaly small in row
        (This heuristic gives much better lower bounds in general.)
        """
        # only use non-slack variables (assumes at least one coefficient equals 1 of higher)
        A = A[:, :A.shape[1] - self.nslack]
        B = np.asmatrix(b) * np.asmatrix(np.ones(A.shape[1])) # B = [b b b ... b]
        min_coeff = np.asmatrix(np.zeros(A.shape)) # initialize X coeff matrix
        
        # case b_i and a_i negative
        all_neg = (B < 0) & (A < 0)
        min_coeff[np.where(all_neg)] = np.divide(B[all_neg],A[all_neg])
        min_coeff[np.where((B >= 0) & (A < 0))] = 0 # invalid
        
        # case where a_i = 0 , x_i either invalid or can be as small as possible
        min_coeff[np.where((B < 0) & (A == 0))] = 0 # invalid
        min_coeff[np.where((B >= 0) & (A == 0))] = 0 # infinitely small

        # bound infinitesimally small coefficients
        A_pos_any = np.asmatrix(np.any(A > 0, axis=1)) * np.asmatrix(np.ones(A.shape[1])).astype(bool)
        min_coeff[A_pos_any] = 0 
        lb = np.max(min_coeff, axis=0) # get current x_i upper bounds
        
        pub = np.nanmax(np.multiply(A, (np.asmatrix(np.ones(A.shape[0])).T * lb)), axis=1) # min_i a_i * x_i for a_i < 0 

        PB = B.copy() # copy neccesary to not rewrite original B matrix
        PB = (B - (pub * np.asmatrix(np.ones(A.shape[1]))))  # pB new b with nb = b - max a_i * x for a_i < 0
        PB[:, 0]

        # repeat same process for new b
        some_neg = (PB < 0) & (A < 0) & (A_pos_any) 
        min_coeff[np.where(some_neg)] = np.divide(PB[some_neg], A[some_neg])
        min_coeff[(PB >= 0) & (A < 0) & (A_pos_any)] = float("inf")

        L = np.min(np.max(min_coeff, axis=0))

        return L
    
    def initial_solution(self, A, b, c, U=None, L=None):
        """Return D and P"""
        # get problem dimensions (Notation from 1510.03339)
        m, n = A.shape[0], A.shape[1]
        
        # if missing: calculate upper and lower bounds using our efficient method
        W_heuristic = self.get_max(A, b)
        if U == None:
            U = np.max([np.max(mat) for mat in [A,b,c]])
            nU = (m+1) * U
            A, b, c = A, b, c
            U = np.max([np.max(mat) for mat in [A,b,c]])
        
        print(f"W actual: {W_heuristic}")
        print(f"W: {2 * (n+1) * ((m+1)*U)**(m+1)}")
        W = np.min([2 * (n+1) * ((m+1)*U)**(m+1), W_heuristic]) # take minimum of paper and our bound
        self.W = W
        
        # if W = 0 then x = 0 otherwise
        if W > 0:
            # calculate L using W (we can change matrix to have only integral values by multiplying it with 1/min)
            min_val = np.min([np.min(np.abs(mat[np.nonzero(mat)])) for mat in [A, b, c] if not (mat is None) and np.any(np.nonzero(mat))])
            min_val = np.min([min_val, 1])
            
            # compare bound using two different W's take maximum (limit to 1e-30 (Note lower bound extremely inefective, similair method to getmax can be used, however not implemented - because there was not a neccesity to do so.))
            L1 = np.min([min_val**(m+1) / (W**2 * 2*n * ((m+1)*U)**(m+1)), 1 / (W**2 * 2*n * ((m+1)*U)**(m+1))]) 
            L2 = np.min([1 / (n / (n+1) * W**2), min_val**(m+1) / (n / (n+1) * W**2)]) if self.L == None else self.L # L can also be treated as precision in which case it can be set
            L3 = self.get_min(A,b) / W
            print(f"L: {L1}")
            print(f"Actual L: {L3}")
            L = np.max([L3, L1, L2, 1e-30]) # 0.24137931 # np.max([L1, L2, 10e-30]) 
            self.L = L
            
            # calculate constants for augmented primal and dual problem
            d = b / W
            e = np.asmatrix(np.ones((n, 1)))
            ro = d - A * e 
            M = 4 * n * U / L # calculate big M

            # augmented primal problem (P') and dual problem (D')
            e_prime = (np.asmatrix(np.ones((1, n+2))))
            e0_prime = (np.asmatrix(np.zeros((m, 1))))
            A_prime = np.r_[np.c_[A, e0_prime, ro], e_prime]
            b_prime = np.r_[d, np.matrix(n+2)]
            c_prime = np.r_[c, np.matrix([0]), np.matrix([M])]

            # initial solution (x0, y0, s0)
            mu = np.squeeze(np.asarray(np.sqrt(c_prime.T * c_prime)))
            x0 = np.asmatrix(np.ones(n+2)).T
            y0 = np.asmatrix(np.r_[np.asmatrix(np.zeros(m)).T, np.matrix([-mu])])
            s0 = np.r_[c + np.ones(c.shape) * mu, np.matrix(np.matrix([mu, M + mu]).T)]
        else:
            # set flag if x = 0 (not to do any more calculations)
            self.stop_flag, self.error_message = True, "x equals 0." 
            return None, None, None, None, None, None, None
        
        return A_prime, b_prime, c_prime, x0, y0, s0, mu 
    
    def solve(self, c, A_eq=None, b_eq=None, A_ub=None, b_ub=None, A_lb=None, b_lb=None, bounds=None, bnd_test=False):
        """
        Solve LP problem min c.T x s.t. A_eq x = b_eq, 
                                    A_ub x <= b_ub,
                                    A_lb x >= b_lb
        all inputs lists, np.arrays or np.matrices
        bounds not used
        boolean:: bnd_test if we're solving bound test (no need to check for boundedness)
        (For our problems we only used A_eq and A_ub, A_lb untested.)
        """
        
        # 0. SAVE INPUTS (for bounded check)
        c_orig = c
        A_ub_orig = A_ub
        
        # I. SETUP PROBLEM
                  
        # specify custom bounds for our problem
        if not bounds is None:
            self.L, self.W = bounds[0], bounds[1]
        else:
            self.L, self.W = None, None
        
        # change to matrix (in case list or np.ndarray)
        A_eq, A_ub, A_lb = np.matrix(A_eq) if not A_eq is None else None, \
                           np.matrix(A_ub) if not A_ub is None else None, \
                           np.matrix(A_lb) if not A_lb is None else None
        b_eq, b_ub, b_lb = np.matrix(b_eq).T if not A_eq is None else None, \
                           np.matrix(b_ub).T if not A_ub is None else None, \
                           np.matrix(b_lb).T if not A_lb is None else None
        clen = len(c)
        c = np.matrix(c)
        
        # remove unnecesary equations (make rows of [A|b] independent)
        if not A_eq is None:
            A_eq, b_eq = self.make_full_rank(A_eq, b_eq)
        if not A_ub is None:
            A_ub, b_ub = self.make_full_rank(A_ub, b_ub)
        if not A_lb is None:
            A_lb, b_lb = self.make_full_rank(A_lb, b_lb)
        
        # add slack variables for inequalities [A I] * [x s].T
        nslack = (A_ub.shape[0] if not A_ub is None else 0) + (A_lb.shape[0] if not A_lb is None else 0)
        self.nslack = nslack
        if not A_ub is None or not A_lb is None:
            A_eq = np.c_[A_eq, np.zeros((A_eq.shape[0], nslack))] if not A_eq is None else None
        A_ub = np.c_[A_ub, np.eye(A_ub.shape[0], nslack)] if not A_ub is None else None
        A_lb = np.c_[A_lb, -np.eye(A_lb.shape[0], nslack, A_ub.shape[0])] if not A_lb is None else None
        c    = np.c_[c, np.zeros((1, nslack))].T if (not A_ub is None or not A_lb is None) else c
        
        # join A, b into oringinal P problem
        A = np.concatenate([Ai for Ai in [A_eq, A_ub, A_lb] if not (Ai is None)], axis=0)
        b = np.concatenate([bi for bi in [b_eq, b_ub, b_lb] if not (bi is None)], axis=1)
        
        # make sure b and c column vectors
        b = b if b.shape[1] == 1 else b.T
        c = c if c.shape[1] == 1 else c.T
        
        # II. get initial solution
        self.A, self.b, self.c, x0, y0, s0, mu0 = self.initial_solution(A,b,c, L=self.lower_bnd, U=self.upper_bnd)
        # if x = 0 just return x = 0
        if self.stop_flag and self.error_message == "x equals 0.":
            return self.Result(x_opt=np.zeros(c.shape),
                           cost=0,
                           is_feasible=True, is_bounded=True)
        
        # III. iterate invariant
        x, y, s = self._solve(self.A, self.b, self.c, x0, y0, s0, mu0)
        
        # get back original x, y, s
        x_opt = np.squeeze(np.asarray(x)[:clen]) * self.W
        y_opt = np.asarray(y)[:(y.shape[0]-1)]
        s_opt = np.squeeze(np.asarray(s)[:clen]) * self.W
        
        # check if is feasible 
        is_feasible = (x[-1] == 0)
        
        #check if is bounded
        is_bounded = (x[-2] == 0)
        if not bnd_test: # we don't care if running for bound check of our original problem
            if not is_bounded: # we don't know wether bounded
                # solve for system minimize c^T x s. t. Ax = 0
                b_bnd_test = [0] * len(b)
                bnd_tester = InteriorPoint(Observer(None, "BTEST"))
                bnd_test_res = bnd_tester.solve(c=c, A_eq=A, b_eq=b_bnd_test, bnd_test=True)
                is_bounded = (bnd_test_res.cost >= 0) # (check if bounded) objective non-negative
            
        return self.Result(x_opt=x_opt,
                           cost=np.sum(np.squeeze(np.asarray(c[:clen])) * x_opt),
                           is_feasible=is_feasible, is_bounded=is_bounded,
                           y_opt=y_opt, s_opt=s_opt)
    
    def _solve(self, A, b, c, x, y, s, mu):
        """
        A,b,c matrices of augmented problem.
        x,y,s initial invariates
        mu initial value gap x_i s_i = mu
        np.matrix:: A, b, c, x, y, s
        float:: mu
        """
        # in case doesnt converge in maxiter
        self.stop_flag, self.error_message = True, "Failed to converge in maxiter."
        
        # get dimensions
        m, n = A.shape[0], A.shape[1]
        
        # set delta by which we decrease value gap mu
        delta = 1 / (8 * np.sqrt(n+2))
        
        # vector of ones [1 1 ... 1]
        e = np.asmatrix(np.ones(n)).T
        
        # get new x, y, s with iterative improvements to value gap mu
        for i in range(self.maxiter):
            # when mu small enought stop
            if mu < self.L**2 / (32 * n**2):
                # specify success
                self.stop_flag, self.error_message = False, "Success."
                
                # get non-zero x and s
                sB = (np.asarray(x).squeeze() >= self.L / (4*m))
                sN = ~sB
                A_B, A_N = A[:, sB], A[:, sN]
                c_B, c_N = c[sB], c[sN]

                if np.linalg.matrix_rank(A_B) != A_B.shape[1]:
                    # if not unique (perturb c and rerun IP method on min c_perturbed.T * x_B s.t A_B * x_B = b)
                    eps = self.L / 2 # eps smaller than L lower bound
                    perturbation = np.matrix([[eps**(1+j) for j in range(A_B.shape[1])]]).T
                    c_prime = c_B + perturbation # c_prime = [c_i + eps**i]
                    
                    # solve perturbed problem
                    psolver = InteriorPoint(Observer(None, "PERTURB"))
                    pres = psolver.solve(c=c_prime, A_eq=A_B, b_eq=b)
                    x_B, y, s_B = np.asmatrix(pres.x).T, pres.y, np.asmatrix(pres.s).T
                else: 
                    # if unique x (solve by gaussian elimination)
                    x_B, _, _, _ = np.linalg.lstsq(A_B, b)
                    y, _, _, _ = np.linalg.lstsq(A_B.T, c_B)
                
                # combine x_N, x_B and s_N, s_B
                s_N = c_N - A_N.T * y

                x[sN, :], x[sB, :] = 0, x_B
                s[sB, :], s[sN, :] = 0, s_N
                break
                
            # calculate new x,y,s by equation given in article
            mu = (1 - delta) * mu
            X, inv_S = np.diag(np.asarray(x).squeeze()), np.diag(1 / np.asarray(s).squeeze())
            k = np.linalg.inv(A * inv_S * X * A.T) * (b - mu * A * inv_S * e)
            f = - A.T * k
            h = - X * inv_S * f + mu * inv_S * e - x
            x = x + h
            y = y + k
            s = s + f  
            
        return x, y, s            

## 2. Diet problem

A man does not live by bread alone - apparently vegetable oil is also needed.

In [32]:
# print-out optimal diet
def display_diet(f_opt, x_opt):
    print(f"min. cost: {round(f_opt, 2)} EUR")
    print(f"CH: {round(np.array([ 18,   48,   5,    1,   5,    0,    0,   8]).dot(x_opt), 2)}")
    print(f"PR: {round(np.array([  2,   11,   3,   13,   3,    0,   15,   1]).dot(x_opt), 2)}")
    print(f"FT: {round(np.array([  0,    5,   3,   10,   3,  100,   30,   1]).dot(x_opt), 2)}")
    print(f"EN: {round(np.array([ 77,  270,  60,  140,  61,  880,  330,  32]).dot(x_opt), 2)}")
    print(20 * "--")
    print(20 * "--")
    foods = ["potatoes", "bread", "milk", "eggs", "yogurt", "veg. oil", "beef", "strawbrs"]
    for food, x in zip(foods, x_opt):
        print(f"{food}: {round(100 * x)}g")

# setup diet LP problem
cost = [10, 22, 15, 45, 40, 20, 87, 21]
constr_A = [[ 18,   48,   5,    1,   5,    0,    0,   8],
            [-18,  -48,  -5,   -1,  -5,    0,    0,  -8],
            [  2,   11,   3,   13,   3,    0,   15,   1],
            [ -2,  -11,  -3,  -13,  -3,    0,  -15,  -1],
            [  0,    5,   3,   10,   3,  100,   30,   1],
            [  0,   -5,  -3,  -10,  -3, -100,  -30,  -1],
            [ 77,  270,  60,  140,  61,  880,  330,  32],
            [-77, -270, -60, -140, -61, -880, -330, -32]]
constr_b = [370, -250, 170, -50, 90, -50, 2400, -2200]

# use scipy solver
print("Scipy.optimize interior point method")
bnd = len(cost) * [(0, float("inf"))]
res = sopt.linprog(c=cost, A_ub=constr_A, b_ub=constr_b, bounds=bnd, method="interior-point")

display_diet(res.fun, res.x)

print()
print(20 * "--")
print(20 * "--")
print()

# use my interior-point method
print("Our interior point method")

ip = InteriorPoint(Observer(None, "IP"))
res = ip.solve(c=cost, A_ub=constr_A, b_ub=constr_b, bounds=None)
print(f"Problem {'' if res.feasible else 'not '}feasible.")
print(f"Problem {'' if res.bounded else 'not '}bounded.")
display_diet(res.cost, res.x)

Scipy.optimize interior point method
min. cost: 148.83 EUR
CH: 299.04
PR: 68.53
FT: 90.0
EN: 2200.0
----------------------------------------
----------------------------------------
potatoes: 0g
bread: 623g
milk: 0g
eggs: 0g
yogurt: 0g
veg. oil: 59g
beef: 0g
strawbrs: 0g

----------------------------------------
----------------------------------------

Our interior point method
W actual: 2400.0
W: 3.4798672548633955e+40
L: 5.3008287973558555e-48
Actual L: 0.0010416666666666667
W actual: 0.0
W: 4.168747121166312e+36
Problem feasible.
Problem bounded.
min. cost: 148.83 EUR
CH: 299.04
PR: 68.53
FT: 90.0
EN: 2200.0
----------------------------------------
----------------------------------------
potatoes: 0g
bread: 623g
milk: 0g
eggs: 0g
yogurt: 0g
veg. oil: 59g
beef: 0g
strawbrs: 0g


/tmp/ipykernel_25900/1155213356.py:120: RuntimeWarning: invalid value encountered in multiply
  nlb = np.nanmin(np.multiply(A, (np.asmatrix(np.ones(A.shape[0])).T * ub)), axis=1) # min_i a_i * x_i for a_i < 0
/tmp/ipykernel_25900/1155213356.py:120: RuntimeWarning: invalid value encountered in multiply
  nlb = np.nanmin(np.multiply(A, (np.asmatrix(np.ones(A.shape[0])).T * ub)), axis=1) # min_i a_i * x_i for a_i < 0


### 2.1. Additional test cases

We picked additional LP-programs from this chanel in order to better test our validity.
Afterall we're of the opinion its fairly easy to overfit on one training sample.
(The simple examples we're borrowed from [Youtube](https://www.youtube.com/watch?v=Y7e7DCsDUMY&ab_channel=Mario%27sMathTutoring)).

In [33]:
cost = [2, 3]
constr_A = [[ -3, -6],
            [-3, -1]]
constr_b = [-24, -9]

# use my interior-point method
print("Our interior point method")
ip = InteriorPoint(Observer(None, "IP"))
res = ip.solve(c=cost, A_ub=constr_A, b_ub=constr_b, bounds=None)
print(f"Problem {'' if res.feasible else 'not '}feasible.")
print(f"Problem {'' if res.bounded else 'not '}bounded.")
print(f"Solution: {res.x}")
print(f"Cost: {round(res.cost)}")

print(20 * "--")

# use scipy solver
print("Scipy.optimize interior point method")
res = sopt.linprog(c=cost, A_ub=constr_A, b_ub=constr_b, method="interior-point")
print(f"Solution: {res.x}")
print(f"Cost: {round(res.fun)}")

Our interior point method
W actual: inf
W: 7290.0
L: 3.2264684896414963e-12
Actual L: 0.0010973936899862826
W actual: inf
W: 7290.0
L: 3.2264684896414963e-12
Actual L: 0.0
Problem feasible.
Problem bounded.
Solution: [2. 3.]
Cost: 13
----------------------------------------
Scipy.optimize interior point method
Solution: [2. 3.]
Cost: 13


/tmp/ipykernel_25900/1155213356.py:120: RuntimeWarning: invalid value encountered in multiply
  nlb = np.nanmin(np.multiply(A, (np.asmatrix(np.ones(A.shape[0])).T * ub)), axis=1) # min_i a_i * x_i for a_i < 0
/tmp/ipykernel_25900/1155213356.py:120: RuntimeWarning: invalid value encountered in multiply
  nlb = np.nanmin(np.multiply(A, (np.asmatrix(np.ones(A.shape[0])).T * ub)), axis=1) # min_i a_i * x_i for a_i < 0


In [34]:
# use my interior-point method
cost = [-10, -20]
constr_A = [[3, 4],
            [-1, 2]]
constr_b = [60, 0]

print("Our interior point method")
ip = InteriorPoint(Observer(None, "IP"))
res = ip.solve(c=cost, A_ub=constr_A, b_ub=constr_b, bounds=None)
print(f"Problem {'' if res.feasible else 'not '}feasible.")
print(f"Problem {'' if res.bounded else 'not '}bounded.")
print(f"Solution: {res.x}")
print(f"Cost: {round(res.cost)}")

print(20 * "--")

# use scipy solver
print("Scipy.optimize interior point method")
res = sopt.linprog(c=cost, A_ub=constr_A, b_ub=constr_b, method="interior-point")
print(f"Solution: {res.x}")
print(f"Cost: {round(res.fun)}")

Our interior point method
W actual: 60.0
W: 58320000.0
L: 5.953741807651272e-12
Actual L: 0.0


/tmp/ipykernel_25900/1155213356.py:120: RuntimeWarning: invalid value encountered in multiply
  nlb = np.nanmin(np.multiply(A, (np.asmatrix(np.ones(A.shape[0])).T * ub)), axis=1) # min_i a_i * x_i for a_i < 0


W actual: 0.0
W: 17280.0
Problem feasible.
Problem bounded.
Solution: [12.  6.]
Cost: -240
----------------------------------------
Scipy.optimize interior point method
Solution: [12.  6.]
Cost: -240


/tmp/ipykernel_25900/1155213356.py:120: RuntimeWarning: invalid value encountered in multiply
  nlb = np.nanmin(np.multiply(A, (np.asmatrix(np.ones(A.shape[0])).T * ub)), axis=1) # min_i a_i * x_i for a_i < 0


## 3. Analytic center (tests on problems with multiple and non-integral solutions)

Interestingly both functions return different optimums (optimums none-the-less), neither of which are analytic centers.

In [35]:
# Analytic center problem 8 (multiple solutions)
cost = [0, 0]
constr_A = [[ -1, 0],
            [0, -1],
            [1, 1],
            [1, 1]]
constr_b = [0, 0, 1, 1]
bnd = len(cost) * [(-float("inf"), float("inf"))]

ip = InteriorPoint(Observer(None, "IP"))
res = ip.solve(c=cost, A_ub=constr_A, b_ub=constr_b, bounds=None)
print("Our interior point method")
print(f"Problem {'' if res.feasible else 'not '}feasible.")
print(f"Problem {'' if res.bounded else 'not '}bounded.")
print(f"Solution: {res.x}")
print(f"Cost: {res.cost}")

print(20 * "--")

print("Scipy.optimize interior point method")
res = sopt.linprog(c=cost, A_ub=constr_A, b_ub=constr_b, bounds=bnd, method="interior-point")
print(f"Solution: {res.x}")
print(f"Cost: {res.fun}")

W actual: 1.0
W: 43750.0
L: 2.6666666666666667e-05
Actual L: 0.0
W actual: 8.0
W: 195689447424.0
L: 1.345085567067707e-23
Actual L: 0.0
W actual: 0.0
W: 746496.0
W actual: 0.0
W: 43750.0
Our interior point method
Problem feasible.
Problem bounded.
Solution: [0. 0.]
Cost: 0.0
----------------------------------------
Scipy.optimize interior point method
Solution: [0.31963437 0.31963437]
Cost: 0.0


/tmp/ipykernel_25900/1155213356.py:120: RuntimeWarning: invalid value encountered in multiply
  nlb = np.nanmin(np.multiply(A, (np.asmatrix(np.ones(A.shape[0])).T * ub)), axis=1) # min_i a_i * x_i for a_i < 0
/tmp/ipykernel_25900/1155213356.py:120: RuntimeWarning: invalid value encountered in multiply
  nlb = np.nanmin(np.multiply(A, (np.asmatrix(np.ones(A.shape[0])).T * ub)), axis=1) # min_i a_i * x_i for a_i < 0


### 3.1 Additional tests

In [36]:
# Analytic center problem 8 (one solution)
cost = [1, 1]
constr_A = [[ -1, 0],
            [0, -1],
            [1, 1],
            [1, 1]]
constr_b = [0, 0, 1, 1]
bnd = len(cost) * [(-float("inf"), float("inf"))]

ip = InteriorPoint(Observer(None, "IP"))
res = ip.solve(c=cost, A_ub=constr_A, b_ub=constr_b, bounds=None)
print("Our interior point method")
print(f"Problem {'' if res.feasible else 'not '}feasible.")
print(f"Problem {'' if res.bounded else 'not '}bounded.")
print(f"Solution: {res.x}")
print(f"Cost: {res.cost}")

print(20 * "--")

print("Scipy.optimize interior point method")
res = sopt.linprog(c=cost, A_ub=constr_A, b_ub=constr_b, bounds=bnd, method="interior-point")
print(f"Solution: {res.x}")
print(f"Cost: {res.fun}")

W actual: 1.0
W: 43750.0
L: 2.6666666666666667e-05
Actual L: 0.0
W actual: 0.0
W: 43750.0
Our interior point method
Problem feasible.
Problem bounded.
Solution: [0. 0.]
Cost: 0.0
----------------------------------------
Scipy.optimize interior point method
Solution: [2.5e-09 2.5e-09]
Cost: 4.999999999997411e-09


/tmp/ipykernel_25900/1155213356.py:120: RuntimeWarning: invalid value encountered in multiply
  nlb = np.nanmin(np.multiply(A, (np.asmatrix(np.ones(A.shape[0])).T * ub)), axis=1) # min_i a_i * x_i for a_i < 0
/tmp/ipykernel_25900/1155213356.py:120: RuntimeWarning: invalid value encountered in multiply
  nlb = np.nanmin(np.multiply(A, (np.asmatrix(np.ones(A.shape[0])).T * ub)), axis=1) # min_i a_i * x_i for a_i < 0


In [37]:
# Analytic center problem 8 (solution smaller than 1)
cost = [-1, -1]
constr_A = [[ -1, 0],
            [0, -1],
            [1, 1],
            [1, 1]]
constr_b = [0, 0, 0.1, 0.5]
bnd = len(cost) * [(-float("inf"), float("inf"))]

ip = InteriorPoint(Observer(None, "IP"))
res = ip.solve(c=cost, A_ub=constr_A, b_ub=constr_b, bounds=None)
print("Our interior point method")
print(f"Problem {'' if res.feasible else 'not '}feasible.")
print(f"Problem {'' if res.bounded else 'not '}bounded.")
print(f"Solution: {res.x}")
print(f"Cost: {res.cost}")

print(20 * "--")

print("Scipy.optimize interior point method")
res = sopt.linprog(c=cost, A_ub=constr_A, b_ub=constr_b, bounds=bnd, method="interior-point")
print(f"Solution: {res.x}")
print(f"Cost: {res.fun}")

/tmp/ipykernel_25900/1155213356.py:120: RuntimeWarning: invalid value encountered in multiply
  nlb = np.nanmin(np.multiply(A, (np.asmatrix(np.ones(A.shape[0])).T * ub)), axis=1) # min_i a_i * x_i for a_i < 0


W actual: 0.5
W: 43750.0
L: 1.066666666666667e-09
Actual L: 0.0
W actual: 8.0
W: 171228266496.0
L: 1.8808563441587556e-180
Actual L: 0.0
W actual: 0.0
W: 653184.0
W actual: 0.0
W: 43750.0
Our interior point method
Problem feasible.
Problem bounded.
Solution: [0.  0.1]
Cost: -0.09999999999999602
----------------------------------------
Scipy.optimize interior point method
Solution: [0.05 0.05]
Cost: -0.1000000000009041


/tmp/ipykernel_25900/1155213356.py:120: RuntimeWarning: invalid value encountered in multiply
  nlb = np.nanmin(np.multiply(A, (np.asmatrix(np.ones(A.shape[0])).T * ub)), axis=1) # min_i a_i * x_i for a_i < 0
